# Web Scraping

In [11]:

from lxml import html
import requests
import requests_html
import pandas as pd
import numpy as np
import re
from selenium import webdriver
from datetime import date,timedelta
from dateutil.parser import parse
import requests
from collections import Counter
import string
import os
from datetime import datetime
from datetime import date
import string


### Ticker Scraping

In [ ]:
alphabet_string = string.ascii_uppercase
alphabet_list = list(alphabet_string)

All_Tickers = []
exchanges = ["NYSE", "NASDAQ"]

for ex in exchanges:
    
    for letter in alphabet_list:  
        
        #create URL and convert html to tree object
        tick_url = 'https://eoddata.com/stocklist/'+ex+'/'+letter+'.htm'  #NASDAQ instead of nyse for nasdaq
        r = requests.get(tick_url)
        tree = html.fromstring(r.content)
        tickers_letter = tree.xpath('//table[@class="quotes"]//tr/td[1]//text()')

        #remove certain tickers
        removeElements = ['COMP', 'DJI', 'SP500', 'DAX', 'FTSE', 'NI225', 'CAC40', 'GLD', 'BDI', 'HSI']
        for element in removeElements: 
            if element in tickers_letter: 
                tickers_letter.remove(element)

        #print(tickers_letter)
        All_Tickers.append(tickers_letter)
        
    #export
    Flat_Tickers = [ticker for sublist in All_Tickers for ticker in sublist]
    Tickers = pd.DataFrame(Flat_Tickers)
    Tickers.to_csv(ex+"_Tickers.csv", index = False)


In [ ]:
#combine both NYSE or NASDAQ
stockslist = pd.DataFrame()
for file in os.listdir("/Users/katherine/Documents/ECON435/Data"): #change directory as needed
    if "Tickers" in file:
        temp_df = pd.read_csv(file)
        stockslist = pd.concat([stockslist, temp_df], ignore_index=True)

In [29]:
#export
stockslist.to_csv("stockslist_1.csv", index = False)

### Xpath Method 1

Both Xpath methods get Daily Discussion as well as What are Your Moves Tommorrow

In [ ]:
########stock list txt import
#with open('readme.txt') as f:
#    lines = f.readlines()
    
#stocks_list = lines

###### OR ##########

#######stock list csv cleaning

#create a list of nyse stock tickers, cleaned
df = pd.read_csv('stockslist_1.csv')
df['sym'] =  [re.findall('[A-Za-z]+', str(x))[0] for x in df.iloc[:, 0]] #use regex to clean symbol formats, some have multiple classes
sym = df.sym.drop_duplicates()
stocks_list = list(sym)

In [9]:
#functions
def get_stock_list(COMMENTS,stocks_list):
    stock_dict = Counter()
    for i in range(len(COMMENTS)):
        for ticker in stocks_list:
            if ticker in COMMENTS[i]:
                    stock_dict[ticker]+=1
    return stock_dict

In [38]:
#set up driver
url = 'https://www.reddit.com/r/wallstreetbets/search/?q=flair%3A%22Daily%20Discussion%22&restrict_sr=1&sort=new'

driver = webdriver.Chrome(executable_path=r'/usr/local/Caskroom/chromedriver/90.0.4430.24/chromedriver')
driver.get(url)


In [39]:
#find thread class
links_path = driver.find_elements_by_xpath('//*[@class="_eYtD2XCVieq6emjKBH3m"]')

DATES = []
LINKS = []

#iterate through links, returns all links and dates in lists
for a in links_path:
    print(a.text) #returns title of thread
    if a.text.startswith('Daily Discussion Thread'):
            date = ' '.join(a.text.split(' ')[-3:])
            print(date)
            DATES.append(date)

            link = a.find_element_by_xpath('../..').get_attribute('href')
            print(link)
            LINKS.append(link)

            stock_link = link.split('/')[-3]
            print(stock_link)
            
    if a.text.startswith('What Are Your Moves Tomorrow'):
            date = ' '.join(a.text.split(' ')[-3:])
            print(date)
            DATES.append(date)

            link = a.find_element_by_xpath('../..').get_attribute('href')
            print(link)
            LINKS.append(link)

            stock_link = link.split('/')[-3]
            print(stock_link)

Daily Discussion Thread for June 04, 2021
June 04, 2021
https://www.reddit.com/r/wallstreetbets/comments/ns1gd8/daily_discussion_thread_for_june_04_2021/
ns1gd8
What Are Your Moves Tomorrow, June 04, 2021
June 04, 2021
https://www.reddit.com/r/wallstreetbets/comments/nrmh2t/what_are_your_moves_tomorrow_june_04_2021/
nrmh2t
What Are Your Moves Tomorrow, June 03, 2021
June 03, 2021
https://www.reddit.com/r/wallstreetbets/comments/nquubf/what_are_your_moves_tomorrow_june_03_2021/
nquubf
Daily Discussion Thread for June 02, 2021
June 02, 2021
https://www.reddit.com/r/wallstreetbets/comments/nqi9f6/daily_discussion_thread_for_june_02_2021/
nqi9f6













About Community
Navigate WSB with Flair


In [40]:
# Make sure comments on each link are sorted by the "top comments"
top_links = [(elem+'?sort=top') for elem in LINKS]


In [41]:
# We tried multiple ways to scrape with xpath. The first was this but, as we can see, it doesnt get many comments
num_comments = []
for url in top_links: 
    session = requests_html.HTMLSession()
    r = session.get(url)
    tree = html.fromstring(r.content)
    comments = tree.xpath('//div[@class = "_292iotee39Lmt0MkQZ2hPV RichTextJSON-root"]//p//text()') #class of comment
    num_comments.append(len(comments))

print(num_comments)

    
# We will not continue with this scrape version - this cell is just to demonstrate that there are different ways 
# to go about it, and not all are necessarily successful 

[23, 26, 32, 48]


### Xpath Method 2

In [42]:
# media":{"richtextContent":{"document":[{"c":[{"e":"text","t":"bbby thank you for today but i am paper handing this right now,"}],"e":"par"}]
session = requests_html.HTMLSession()
FULL_TICK_LIST = []

for url in top_links: 
    r = session.get(url)
    from lxml import html
    tree = html.fromstring(r.content)
    comments = tree.xpath('//script[@id="data"]//text()')
    
    my_str = str(comments[0])
    #newlist = [str(s) for s in comments[0]]
    #map(str, comments[0])
    comment_list = my_str.split('"e":"text","t":', -1)
    comment_list[0].split("}",1)[0]

    COMMENTS = []
    for i in range(1,len(comment_list)):
            commt = comment_list[i].split("}",1)[0]
            COMMENTS.append(commt)
            
    stock_dict = get_stock_list(COMMENTS,Flat_Tickers)
    FULL_TICK_LIST.append(stock_dict)

print(len(FULL_TICK_LIST))
driver.close()

4


In [44]:
final_agg = {}  

#convert to dictionary
def clean_agg(agg):
    cleaned_agg = {k:v for k,v in agg.items()} #not used: if ((v > 11) or (len(k) > 1))}
    return cleaned_agg

for i in range(len(FULL_TICK_LIST)):
    cleaned_day_agg = clean_agg(FULL_TICK_LIST[i])
    final_agg[DATES[i]]= cleaned_day_agg
    
len(FULL_TICK_LIST)

4

In [45]:
#convert to df 
final_df = pd.DataFrame.from_dict(final_agg, orient='index') 
final_df

,B,BB,H,K,A,L,T,Y,O,J,...,FF,TAK,OIS,HES,MHO,FI,NNA,SAVE,WELL,IP
"June 04, 2021",185,170,42,11,58,43,48,28,36,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"June 03, 2021",188,170,27,6,62,40,42,13,29,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"June 02, 2021",188,160,31,8,58,29,43,18,43,8,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


This method still doesn't get enough comments. We use the Pushshift API next.

### API Method

In [ ]:
#import sys
#!{sys.executable} -m pip install requests_html

In [4]:
########stock list txt import
#with open('readme.txt') as f:
#    lines = f.readlines()
    
#stocks_list = lines

###### OR ##########

#######stock list csv cleaning

#create a list of nyse stock tickers, cleaned
df = pd.read_csv('stockslist_1.csv')
df['sym'] =  [re.findall('[A-Za-z]+', str(x))[0] for x in df.iloc[:, 0]] #use regex to clean symbol formats, some have multiple classes
sym = df.sym.drop_duplicates()
stocks_list = list(sym)

In [36]:
#get selenium set up with the needed URL
url = 'https://www.reddit.com/r/wallstreetbets/search/?q=flair%3A%22Daily%20Discussion%22&restrict_sr=1&sort=new'

driver = webdriver.Chrome(executable_path=r'/usr/local/Caskroom/chromedriver/90.0.4430.24/chromedriver')
driver.get(url)


In [37]:
#scroll down in selenium browser until the dates you want to go back until shows up in the html
#then the desired days will show up in links
#note: as you scroll down the more recent dates disappear from the html

links = driver.find_elements_by_xpath('//*[@class="_eYtD2XCVieq6emjKBH3m"]') #this class gets title of post
print(links)

#print all links in the driver
for a in links:
        print(a.text)


[<selenium.webdriver.remote.webelement.WebElement (session="019cbee8c2596424c16079545fce3298", element="60c86f8f-b4be-4aa3-8ad6-653ce6c08f25")>, <selenium.webdriver.remote.webelement.WebElement (session="019cbee8c2596424c16079545fce3298", element="7daad3c1-1226-447b-aa40-e1327a199db7")>, <selenium.webdriver.remote.webelement.WebElement (session="019cbee8c2596424c16079545fce3298", element="671f6376-5df5-4da8-8e39-bc1f631ef656")>, <selenium.webdriver.remote.webelement.WebElement (session="019cbee8c2596424c16079545fce3298", element="47b2e04d-b973-4641-80b8-ffc66b6d2ebd")>, <selenium.webdriver.remote.webelement.WebElement (session="019cbee8c2596424c16079545fce3298", element="3c212181-99a4-4696-b2de-4675864ce558")>, <selenium.webdriver.remote.webelement.WebElement (session="019cbee8c2596424c16079545fce3298", element="1d15ad14-7a79-49c5-b4e0-0b702ddd103a")>, <selenium.webdriver.remote.webelement.WebElement (session="019cbee8c2596424c16079545fce3298", element="f10b9e46-c194-4da3-b95c-d4c44467






























What Are Your Moves Tomorrow, April 30, 2021
Daily Discussion Thread for April 29, 2021
What Are Your Moves Tomorrow, April 29, 2021
Daily Discussion Thread for April 28, 2021
What Are Your Moves Tomorrow, April 28, 2021
Daily Discussion Thread for April 27, 2021
What Are Your Moves Tomorrow, April 27, 2021
Daily Discussion Thread for April 26, 2021
What Are Your Moves Tomorrow, April 26, 2021
Daily Discussion Thread for April 23, 2021
What Are Your Moves Tomorrow, April 23, 2021
Daily Discussion Thread for April 22, 2021
What Are Your Moves Tomorrow, April 22, 2021
Daily Discussion Thread for April 21, 2021
What Are Your Moves Tomorrow, April 21, 2021
Daily Discussion Thread for April 20, 2021
What Are Your Moves Tomorrow, April 20, 2021
Daily Discussion Thread for April 19, 2021
What Are Your Moves Tomorrow, April 19, 2021
Daily Discussion Thread for April 16, 2021
What Are Your Moves Tomorrow, April 16, 2021
Daily Discussion Thread for April 15, 2021
Wha

In [9]:
########functions

def grab_link_and_comments(links, yesterday):
    raw_comment_list = {}
    
    #iterate through list and find the link that matches with the date specificed and is a Daily Discussion using xpath
    for a in links:
        if a.text.startswith('Daily Discussion Thread'):
            date = ''.join(a.text.split(' ')[-3:])
            print(date)
            parsed = parse(date) 
            print(parsed)
            print(parse(str(yesterday)))
            if parse(str(yesterday)) == parsed:
                link = a.find_element_by_xpath('../..').get_attribute('href')
                print(link)
                stock_link = link.split('/')[-3] # stock link contains the thread id to pull comments
                html = requests.get(f'https://api.pushshift.io/reddit/submission/comment_ids/{stock_link}')
                raw_comment_list = html.json() # contains all comment id's for a particular thread
    return raw_comment_list
    #example of raw_comment_list
    #['h06doqh', 'h06doyx', 'h06dp6j', 'h06dpkc', 'h06dpkn', 'h06dppa'... 'h06dt7g', 'h06dth4', 'h06dtpm', 'h06dugg']

# def clean_agg(agg):
#     cleaned_agg = {k:v for k,v in agg.items() if ((v > 11) or (len(k) > 1))}
#     return cleaned_agg

def get_comments(comment_list):
    #use reddit api to get the list of text for the comments
    html = requests.get(f'https://api.pushshift.io/reddit/comment/search?ids={comment_list}&fields=body&size=1000')
    newcomments = html.json()
    return newcomments

def get_stock_list(newcomments,stocks_list):
    stock_dict = Counter()
    
    #for each comment text, count the number of times the ticker symbols appear
    for a in newcomments['data']: 
        for ticker in stocks_list:
            if ticker in a['body']:
                    stock_dict[ticker]+=1
    return stock_dict





In [38]:
##################main
    
date_change = np.arange(35,42) #set the desired date range, had to do in chunks due to api
remove_me = slice(0,100) #since reddit api only allows 100 queries at a time

final_agg = {} #aggregation dictionary of all dates/threads

#iterate through all relevant dates
for d in date_change:
    d = int(d)
    yesterday = date.today() - timedelta(days=d) #creates the date object you are interested in 
    raw_comment_list = grab_link_and_comments(links, yesterday)
    if raw_comment_list:
        print(raw_comment_list.keys())
        orig_list = np.array(raw_comment_list['data'])
        if len(orig_list) > 20000: # we chose to cap at 20000 due to some finicky errors from the api
            orig_list = orig_list[0:20000]
        day_agg = Counter() #create an aggregate counter for the whole day/thread
        i = 0 #create an iterator for error checking
        while len(orig_list)>0:
            print(i)
            #we split into 100 each time and keep adding to the day_agg counter
            if len(orig_list)>100:
                comment_list = ",".join(orig_list[0:100])
                newcomments = get_comments(comment_list)
                orig_list = np.delete(orig_list, remove_me)
                stock_dict = get_stock_list(newcomments,stocks_list)
                day_agg = day_agg + stock_dict
            else:
                comment_list = ",".join(orig_list[0:])
                newcomments = get_comments(comment_list)
                stock_dict = get_stock_list(newcomments,stocks_list)
                day_agg = day_agg + stock_dict
                break
            i = i+1
        
        #check and add to final aggregated dictionary
        yes = str(yesterday)
        if yes in final_agg:
            continue
        else:
            final_agg[yes] = day_agg
    else:
        continue
    
    
driver.close()


April29,2021
2021-04-29 00:00:00
2021-04-29 00:00:00
https://www.reddit.com/r/wallstreetbets/comments/n11379/daily_discussion_thread_for_april_29_2021/
n11379
April28,2021
2021-04-28 00:00:00
2021-04-29 00:00:00
April27,2021
2021-04-27 00:00:00
2021-04-29 00:00:00
April26,2021
2021-04-26 00:00:00
2021-04-29 00:00:00
April23,2021
2021-04-23 00:00:00
2021-04-29 00:00:00
April22,2021
2021-04-22 00:00:00
2021-04-29 00:00:00
April21,2021
2021-04-21 00:00:00
2021-04-29 00:00:00
April20,2021
2021-04-20 00:00:00
2021-04-29 00:00:00
April19,2021
2021-04-19 00:00:00
2021-04-29 00:00:00
April16,2021
2021-04-16 00:00:00
2021-04-29 00:00:00
April15,2021
2021-04-15 00:00:00
2021-04-29 00:00:00
April14,2021
2021-04-14 00:00:00
2021-04-29 00:00:00
April13,2021
2021-04-13 00:00:00
2021-04-29 00:00:00
April12,2021
2021-04-12 00:00:00
2021-04-29 00:00:00
April09,2021
2021-04-09 00:00:00
2021-04-29 00:00:00
April08,2021
2021-04-08 00:00:00
2021-04-29 00:00:00
April07,2021
2021-04-07 00:00:00
2021-04-29 00

April15,2021
2021-04-15 00:00:00
2021-04-25 00:00:00
April14,2021
2021-04-14 00:00:00
2021-04-25 00:00:00
April13,2021
2021-04-13 00:00:00
2021-04-25 00:00:00
April12,2021
2021-04-12 00:00:00
2021-04-25 00:00:00
April09,2021
2021-04-09 00:00:00
2021-04-25 00:00:00
April08,2021
2021-04-08 00:00:00
2021-04-25 00:00:00
April07,2021
2021-04-07 00:00:00
2021-04-25 00:00:00
April06,2021
2021-04-06 00:00:00
2021-04-25 00:00:00
April05,2021
2021-04-05 00:00:00
2021-04-25 00:00:00
April01,2021
2021-04-01 00:00:00
2021-04-25 00:00:00
March31,2021
2021-03-31 00:00:00
2021-04-25 00:00:00
March30,2021
2021-03-30 00:00:00
2021-04-25 00:00:00
April29,2021
2021-04-29 00:00:00
2021-04-24 00:00:00
April28,2021
2021-04-28 00:00:00
2021-04-24 00:00:00
April27,2021
2021-04-27 00:00:00
2021-04-24 00:00:00
April26,2021
2021-04-26 00:00:00
2021-04-24 00:00:00
April23,2021
2021-04-23 00:00:00
2021-04-24 00:00:00
April22,2021
2021-04-22 00:00:00
2021-04-24 00:00:00
April21,2021
2021-04-21 00:00:00
2021-04-24 00

In [345]:
final_agg

{'2021-05-28': {'H': 2538,
  'F': 1685,
  'FI': 40,
  'IR': 35,
  'R': 2594,
  'RS': 74,
  'ST': 292,
  'T': 5039,
  'A': 6584,
  'AM': 4354,
  'AMC': 4213,
  'C': 5824,
  'E': 3140,
  'G': 2640,
  'GM': 887,
  'GME': 874,
  'M': 6387,
  'MC': 4266,
  'W': 2500,
  'J': 810,
  'AB': 98,
  'B': 4094,
  'BA': 140,
  'BY': 160,
  'Y': 2599,
  'PM': 31,
  'AL': 171,
  'ALL': 95,
  'L': 3742,
  'LL': 268,
  'BB': 1429,
  'CE': 254,
  'SP': 312,
  'SPCE': 186,
  'BL': 126,
  'BLX': 49,
  'BYND': 83,
  'D': 2838,
  'FUBO': 22,
  'GO': 263,
  'GOOD': 6,
  'K': 993,
  'LU': 33,
  'LX': 51,
  'O': 2836,
  'RBLX': 49,
  'TS': 284,
  'TSLA': 68,
  'U': 1219,
  'X': 205,
  'V': 761,
  'CC': 45,
  'DE': 89,
  'ID': 72,
  'IDE': 28,
  'RIDE': 20,
  'EOD': 105,
  'ED': 118,
  'UE': 32,
  'FOMO': 101,
  'MO': 389,
  'OM': 197,
  'NOK': 135,
  'TC': 28,
  'TT': 38,
  'MOR': 28,
  'OR': 201,
  'RE': 252,
  'MOV': 12,
  'MOVE': 7,
  'PLTR': 381,
  'TR': 488,
  'CL': 104,
  'ET': 299,
  'HI': 149,
  'KE': 1

In [39]:
#########clean up df

#convert to df
final_df = pd.DataFrame.from_dict(final_agg, orient='index') 

#take out single letter tickers
keep = []
for name in final_df.columns:
    if len(name) > 1:
        keep.append(name)
        
final = final_df[keep]

#take out data with 10 or less counts
fin = final.mask(final < 11)

#export
fin = fin.dropna(axis=1, how='all')
fin.to_csv('Days35-42.csv') #change name according to date range


In [3]:
###############after all data has been exported to csv's
#concatenate all relevant files to get complete dataset

agg_df = pd.DataFrame()
for file in os.listdir("/Users/katherine/Documents/ECON435"): #change as needed
    if file.startswith("Days"):
        temp_df = pd.read_csv(file)
        agg_df = pd.concat([agg_df,temp_df], axis=0, ignore_index=True)
        
agg_df = agg_df.sort_values(by = ['Unnamed: 0'])
agg_df.rename(columns={'Unnamed: 0': "Date"}, errors="raise", inplace = True)

agg_df

    

,Date,AM,AMD,MD,PLTR,TR,TA,AA,AAP,AAPL,...,LMND,RMO,RAD,NGS,COP,TTD,EARN,APPS,ABR,SABR
10,2021-04-23,223,143,147,139,148,30,123,115,115,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2021-04-26,361,127,203,143,155,14,90,83,83,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2021-04-27,247,138,979,122,134,22,82,73,73,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2021-04-28,492,359,692,99,109,27,146,136,134,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2021-04-29,262,91,129,122,142,19,234,222,222,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-04-30,279,125,148,82,142,18,152,141,141,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-05-03,411,304,327,169,247,16,56,45,45,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-05-04,248,161,187,202,220,28,167,152,151,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2021-05-05,294,188,207,234,260,18,90,63,63,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,2021-05-06,302,152,179,575,612,58,67,45,45,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#######final export
agg_df.to_csv('NYSE days 0-42.csv')

In [52]:
##########can manually choose ticker symbols to be analyzed with financial data
to_analyze = pd.read_csv('NYSE days 0-42.csv')
analyze_list = ['Date','AMD','PLTR','TSLA','AMC','AAPL','GME','NIO','MVIS','CLOV','VIAC','WMC','NVDA','OCGN','BABA','BB','NOK']


to_analyze_subset = to_analyze[[c for c in to_analyze.columns if c in analyze_list]]
to_analyze_subset = to_analyze_subset.fillna(0)

to_analyze_subset.to_csv('NYSE days 0-42 to_analyze.csv')